<a href="https://colab.research.google.com/github/mohmiim/MLIntroduction/blob/master/session-4/Session_4_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)


2.2.0-rc1


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import xception
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout, Conv2D, Flatten, MaxPool2D
from tensorflow.keras.activations import relu,softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [0]:
# data = > https://drive.google.com/file/d/1GEKK8oRNntFyR0ZxPdcvPut-15b7CvrW/view?usp=sharing
# small Data => https://drive.google.com/file/d/1OHGNsTfvVZvWYQ7B29SYcxrLGVdeCoQb/view?usp=sharing




In [3]:
import os

!wget --no-check-certificate \
    https://github.com/mohmiim/MLIntroduction/raw/master/data/data.zip \
    -O /tmp/data.zip

--2020-03-29 15:34:01--  https://github.com/mohmiim/MLIntroduction/raw/master/data/data.zip
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mohmiim/MLIntroduction/master/data/data.zip [following]
--2020-03-29 15:34:07--  https://raw.githubusercontent.com/mohmiim/MLIntroduction/master/data/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19729033 (19M) [application/zip]
Saving to: ‘/tmp/data.zip’

/tmp/data.zip       100%[===================>]  18.81M  89.9MB/s    in 0.2s    

2020-03-29 15:34:07 (89.9 MB/s) - ‘/tmp/data.zip’ saved [19729033/19729033]



In [0]:

import zipfile
def unzip(file,target):
  zip_ref = zipfile.ZipFile(file, 'r')
  zip_ref.extractall(target)
  zip_ref.close()

unzip("/tmp/data.zip","/tmp")

In [5]:
print(os.listdir("/tmp/data"))


['test', 'train', '.gitignore']


In [0]:
#default sizes
Image_Width = 100
Image_Height = 100
Image_Depth = 3
targetSize = (Image_Width,Image_Height)
targetSize_withdepth = (Image_Width,Image_Height,Image_Depth)

epochs = 300
x_train = []
y_train = []
y_labels = []

In [7]:
#define the sub folders for both training and test
training = os.path.join("/tmp/data",'train')

#now the easiest way to load data is to use the ImageDataGenerator
train_data_generator = ImageDataGenerator(preprocessing_function=xception.preprocess_input,
                                          width_shift_range=0.2,
                                          height_shift_range=0.2,
                                          zoom_range=0.2,
                                          fill_mode='nearest')


train_generator = train_data_generator.flow_from_directory(training,
                                                           batch_size=229,
                                                           target_size=targetSize,
                                                           #seed=12
                                                           shuffle=False
                                                           )

Found 229 images belonging to 5 classes.


In [8]:
y_train =  train_generator.classes
for k in train_generator.class_indices.keys():
    y_labels.append(k)
y_train = to_categorical(y_train)

print(len(y_train))
print(len(y_labels))

229
5


In [9]:
# NOW WE LOAD THE PRE_TRAINED MODEL
FEATURE_EXTRACTOR = xception.Xception(weights='imagenet',include_top=False,input_shape=targetSize_withdepth)

#now we can extract the feature and save them for our images
features_x = FEATURE_EXTRACTOR.predict(train_generator,steps=1)

83689472/83683744 [==============================] - 1s 0us/step


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [10]:
model = Sequential()

#add our layers
model.add(Flatten(input_shape=features_x.shape[1:]))
model.add(Dense(128,activation=relu))
model.add(Dropout(0.1))
model.add(Dense(64,activation=relu))
model.add(Dense(len(y_labels),activation='softmax'))
history = model.compile(optimizer=Adam(lr=0.0001), loss="categorical_crossentropy", metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2359424   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
Total params: 2,368,005
Trainable params: 2,368,005
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(features_x,y_train,epochs=epochs,shuffle=True,verbose=2)

In [12]:
test_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data_generator.flow_from_directory("/tmp/data/test",
                                                         target_size=(100,100),
                                                         batch_size=93,
                                                         shuffle=False)

Found 93 images belonging to 5 classes.


In [13]:
import numpy as np
x_train = []
y_train = []
y_labels = {}
x_train = np.array(x_train)
print(y_labels)
#normalize the image data same way  as the model
x_train = xception.preprocess_input(x_train)

#now we can extract the feature and save them for our images
features_x = FEATURE_EXTRACTOR.predict(test_generator, steps=1)
predictions = model.predict(features_x)

{}


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [14]:
from sklearn.metrics import confusion_matrix,classification_report
row_index = predictions.argmax(axis=1)
filenames = test_generator.filenames
nb_samples = len(filenames)
y_true = test_generator.classes
target_names = test_generator.class_indices.keys()
print(target_names)
print(confusion_matrix(y_true, row_index))
print('Classification Report')
target_names = test_generator.class_indices.keys()
print(classification_report(test_generator.classes, row_index, target_names=target_names))

dict_keys(['bar_chart', 'bubble_chart', 'pie_chart', 'radar_chart', 'treemap_chart'])
[[16  1  0  0  2]
 [ 0 19  0  0  0]
 [ 1  0 17  0  0]
 [ 0  0  2 16  0]
 [ 1  1  0  0 17]]
Classification Report
               precision    recall  f1-score   support

    bar_chart       0.89      0.84      0.86        19
 bubble_chart       0.90      1.00      0.95        19
    pie_chart       0.89      0.94      0.92        18
  radar_chart       1.00      0.89      0.94        18
treemap_chart       0.89      0.89      0.89        19

     accuracy                           0.91        93
    macro avg       0.92      0.91      0.91        93
 weighted avg       0.92      0.91      0.91        93

